In [96]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re

# lyrics text to array

In [2]:
lyrics_full = pd.read_table('lyrics_full.txt', header = None)
lyrics_jamo = pd.read_table('lyrics_jamo.txt', header = None)

In [3]:
lyrics_full = np.array(lyrics_full[0])
lyrics_jamo = np.array(lyrics_jamo[0])

In [4]:
lyrics_full_20_sep = []

for sentence in lyrics_full:
    if len(sentence) <= 20:
        sep_list = list(sentence)
        while len(sep_list) < 20:
            sep_list.append('<eos>')
    lyrics_full_20_sep.append(sep_list)

In [5]:
lyrics_jaeum_20_sep = []

for sentence in lyrics_jamo:
    if len(sentence) <= 20:
        sep_list = list(sentence)
        while len(sep_list) < 20:
            sep_list.append('<eos>')
    lyrics_jaeum_20_sep.append(sep_list)

In [6]:
full_lyrics_20_sep = []

for ls in lyrics_full_20_sep:
    full_lyrics_20_sep += ls

In [7]:
jaeum_lyrics_20_sep = []

for ls in lyrics_jaeum_20_sep:
    jaeum_lyrics_20_sep += ls

In [8]:
full_part = full_lyrics_20_sep[:500000]
jaeum_part = jaeum_lyrics_20_sep[:500000]

## lyrics one_hot encoding

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import hgtk
import collections

## full_lyrics

### LabelEncoder

In [10]:
full_counter = collections.Counter(full_part)
#print(full_counter)

In [11]:
full_count_pairs = sorted(full_counter.items(), key=lambda x: -x[1])
#print(full_count_pairs)

In [12]:
full_chars, _ = zip(*full_count_pairs)
#print(full_chars)

In [13]:
jaeum_counter = collections.Counter(jaeum_part)
jaeum_count_pairs = sorted(jaeum_counter.items(), key=lambda x: -x[1])
jaeum_chars, _ = zip(*jaeum_count_pairs)
jaeum_vocab_dict = dict(zip(jaeum_chars, range(len(jaeum_chars))))

In [14]:
full_vocab_size = len(full_chars)
full_vocab_dict = dict(zip(full_chars[2:], range(21,len(full_chars)+21)))
#print(full_vocab_dict)

In [15]:
vocab_dict = jaeum_vocab_dict.copy()
vocab_dict.update(full_vocab_dict)
inv_vocab = {v: k for k, v in vocab_dict.items()}

### OneHotEncoder

In [20]:
interger_encoded_full_vocab_values = interger_encoded_full_vocab_values.reshape(len(interger_encoded_full_vocab_values), 1)

In [21]:
enc = OneHotEncoder()

In [22]:
full_vocab_embedded = enc.fit_transform(interger_encoded_full_vocab_values).toarray()

In [23]:
full_vocab_embedded = full_vocab_embedded.astype(np.float32)

## jaeum_lyrics

### LabelEncoder

In [27]:
jaeum_counter = collections.Counter(jaeum_lyrics_50_sep)
#print(jaeum_counter)

In [28]:
jaeum_count_pairs = sorted(jaeum_counter.items(), key=lambda x: -x[1])
#print(jaeum_count_pairs)

In [29]:
jaeum_chars, _ = zip(*jaeum_count_pairs)
#print(jaeum_chars)

In [30]:
jaeum_vocab_size = len(jaeum_chars)
jaeum_vocab_dict = dict(zip(jaeum_chars, range(len(jaeum_chars))))
#print(jaeum_vocab_dict)

In [31]:
interger_encoded_jaeum_vocab_values = np.array(list(jaeum_vocab_dict.values()))

In [32]:
interger_encoded_jaeum_vocab_values

### OneHotEncoder

In [23]:
interger_encoded_vocab_values = np.array(list(vocab_dict.values()))

In [24]:
interger_encoded_vocab_values = interger_encoded_vocab_values.reshape(len(interger_encoded_vocab_values), 1)

In [25]:
enc = OneHotEncoder()

In [26]:
vocab_embedded = enc.fit_transform(interger_encoded_vocab_values).toarray()

In [27]:
vocab_embedded = vocab_embedded.astype(np.float32)

In [41]:
jaeum_vocab_embedded = enc.fit_transform(interger_encoded_jaeum_vocab_values).toarray()

In [42]:
jaeum_vocab_embedded = jaeum_vocab_embedded.astype(np.float32)

In [44]:
# 자음
# 음절 -> 라벨 dict
#jaeum_vocab_dict

In [45]:
# 자음
# 라벨 -> one hot vector embedding
#jaeum_vocab_embedded

## Modeling

## Batching

In [41]:
batch_size = 100
seq_length = 20
vocab_size = len(vocab_dict.keys())
num_batches = len(full_part) / (batch_size*seq_length)

In [49]:
for i in range(100,len(full_part)):
    if len(full_part)/i%seq_length==0:
        print(i,len(full_part)/(i*seq_length))

100 250.0
125 200.0
200 125.0
250 100.0
500 50.0
625 40.0
1000 25.0
1250 20.0
2500 10.0
3125 8.0
5000 5.0
6250 4.0
12500 2.0
25000 1.0


In [32]:
def create_batches(chosung_data, full_data, batch_size, seq_length):
    xdata = np.array(list(map(vocab_dict.get, chosung_data)))
    ydata = np.array(list(map(vocab_dict.get, full_data)))
    
    x_reshape = xdata.reshape(batch_size, -1)
    y_reshape = ydata.reshape(batch_size, -1)
    
    x_batches = np.split(x_reshape, x_reshape.shape[1]/seq_length, 1)
    y_batches = np.split(y_reshape, y_reshape.shape[1]/seq_length, 1)
   
    return x_batches, y_batches
    
pointer = 0 
def next_batch(x_batches, y_batches):    
    global pointer
    x, y = x_batches[pointer], y_batches[pointer]
    pointer += 1
    return x, y

def reset_batch_pointer():
    global pointer
    pointer = 0

## Modeling

In [101]:
# Define Network
rnn_size = vocab_size
num_layers = 3
grad_clip = 5.


MODEL_PATH = 'small models'
LOGS_PATH = 'small logs'


class chosungRNN:

    def __init__(self, tf, batch_size, seq_length, vocab_size, vocab_embedded, start_learning_rate, decay_step, decay_rate, dropout_rate,is_training):
        tf.reset_default_graph()
        self.tf = tf
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.vocab_size = vocab_size

        # Select RNN Cell
        unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
        if is_training :
            cell = tf.contrib.rnn.DropoutWrapper(unitcell, output_keep_prob= 1- dropout_rate)
            self.cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
        else:
            self.cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)
                

        # Set paths to the graph
        self.input_data = tf.placeholder(tf.int32, [None, self.seq_length], name='input_data')
        self.targets = tf.placeholder(tf.int32, [None, self.seq_length], name='targets')
        self.initial_state = self.cell.zero_state(batch_size, tf.float32)

        # Set Network
        with tf.variable_scope('rnnlm')  :
            
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
            softmax_b = tf.get_variable("softmax_b", [vocab_size])
            

            self.embedding = vocab_embedded

            inputs = tf.split(tf.nn.embedding_lookup(self.embedding, self.input_data), self.seq_length, 1)
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

        # Output of RNN       
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(
            inputs, self.initial_state, self.cell, loop_function=None, scope='rnnlm')
        output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

        # Next word probability
        self.probs = tf.nn.softmax(logits)

        # Define LOSS
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits],  # Input
                                                                  [tf.reshape(self.targets, [-1])],  # Target
                                                                  [tf.ones([self.batch_size * self.seq_length])],  # Weight
                                                                  self.vocab_size)

        # Define Optimizer
        self.cost = tf.reduce_sum(loss) / self.batch_size / self.seq_length
        tf.summary.scalar("cost", self.cost)

        self.final_state = last_state
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), grad_clip)

        # for weight decay
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, decay_step, decay_rate, staircase=False)
        tf.summary.scalar("learning_rate", learning_rate)

        self.optm = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, tvars))

        self.merged = tf.summary.merge_all()

        # make directories
        if not os.path.exists(MODEL_PATH):
            os.mkdir(MODEL_PATH)

        if not os.path.exists(LOGS_PATH):
            os.mkdir(LOGS_PATH)

        # init session
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        # saver
        self.saver = tf.train.Saver(tf.global_variables())
        self.summary_writer = tf.summary.FileWriter(LOGS_PATH, graph=self.sess.graph)

    def restore(self):
        latest_checkpoint = self.tf.train.latest_checkpoint(MODEL_PATH)

        if not latest_checkpoint:
            print("Trained network not found on ", MODEL_PATH)
            return False
        
        self.saver.restore(self.sess, latest_checkpoint)
        return True

    def save(self, step):
        checkpoint_path = os.path.join(MODEL_PATH, 'model.ckpt')
        self.saver.save(self.sess, checkpoint_path, global_step=step)
        print("model saved to {}".format(checkpoint_path))
        
    def get_state(self):
        return self.sess.run(self.initial_state)

    def train(self, x, y, last_state, step):
        train_loss, state, _, summary = self.sess.run(
            [self.cost, self.final_state, self.optm, self.merged], {
                self.input_data: x,
                self.targets: y,
                self.initial_state: last_state
            })
        self.summary_writer.add_summary(summary, step)
        return train_loss, state

    # Sampling function
    
    def sample(self, prime):
        state = self.sess.run(self.cell.zero_state(1, self.tf.float32))
        
        prime_list = list(prime)
        if len(prime_list) != self.seq_length :
            while len(prime_list) != self.seq_length:
                prime_list.append('<eos>')

        x = np.zeros((1, self.seq_length)).tolist()[0]
        for i, char in enumerate(prime_list):
            x[i] = vocab_dict[char]
        x = [x]
        feed = {self.input_data: x, self.initial_state: state}
        [state] = self.sess.run([self.final_state], feed)
        

        
        def weighted_pick(weight):
            t = np.cumsum(weight)
            s = np.sum(weight)
            return int(np.searchsorted(t, np.random.rand(1) * s))
            
        pred = ''
        x = np.zeros((1, self.seq_length)).tolist()[0]
        for n in range(len(prime_list)):
            x[n] = vocab_dict[prime_list[n]]
        x = [x]
        feed = {self.input_data: x, self.initial_state: state}
        
        [_probsval, state] = self.sess.run([self.probs, self.final_state], feed)
        
        sample_list = []
            # sample = int(np.random.choice(len(p), p=p))
        for pro in _probsval:
            sample_list.append(weighted_pick(pro))
            
        for sample in sample_list:
            pred += inv_vocab.get(sample)
        
        return pred

## Training

In [34]:
#-*- coding: utf-8 -*-

from __future__ import print_function
from __future__ import unicode_literals

import tensorflow as tf
from tqdm import trange


### Build Network

In [42]:
LEARNING_RATE = 0.002
DECAY_STEP = num_batches
DECAY_RATE = 0.97
DROPOUT_RATE = 0.3



net = chosungRNN(tf, batch_size, seq_length, vocab_size, vocab_embedded, LEARNING_RATE, num_batches, DECAY_RATE, DROPOUT_RATE, True)
print("Network Ready")

Network Ready


### Training

In [48]:
chosung_data = jaeum_part
full_data = full_part

In [49]:
# Train the model!

NUM_EPOCHS = 50
SAVE_EVERY = 100
pointer = 0

x, y = create_batches(chosung_data, full_data,batch_size, seq_length)

for e in trange(NUM_EPOCHS):  # for all epochs

    reset_batch_pointer()
    num_batches = int(num_batches)
    state = net.get_state()
    for b in range(num_batches):
        
        step = e * num_batches + b
        next_x, next_y = next_batch(x, y)
        
        # Train!
        train_loss, state = net.train(next_x, next_y, state, step)
        
        if b % 100 == 0:
            print("{}/{} (epoch {}), train_loss = {:.8f}".format(
                step,
                NUM_EPOCHS * num_batches,
                e, train_loss))

        if step % SAVE_EVERY == 0:
            net.save(step=step)

  0%|          | 0/50 [00:00<?, ?it/s]

0/12500 (epoch 0), train_loss = 1.19744623
model saved to small models/model.ckpt
100/12500 (epoch 0), train_loss = 0.77311051
model saved to small models/model.ckpt
200/12500 (epoch 0), train_loss = 0.78535289
model saved to small models/model.ckpt


  2%|▏         | 1/50 [06:03<4:56:45, 363.37s/it]

250/12500 (epoch 1), train_loss = 1.02527857
model saved to small models/model.ckpt
350/12500 (epoch 1), train_loss = 0.70082772
model saved to small models/model.ckpt
450/12500 (epoch 1), train_loss = 0.68719113


  4%|▍         | 2/50 [12:03<4:49:13, 361.53s/it]

...

 96%|█████████▌| 48/50 [4:50:14<12:05, 362.81s/it]

12000/12500 (epoch 48), train_loss = 0.18621527
model saved to small models/model.ckpt
12100/12500 (epoch 48), train_loss = 0.10107493
model saved to small models/model.ckpt
12200/12500 (epoch 48), train_loss = 0.11163632
model saved to small models/model.ckpt


 98%|█████████▊| 49/50 [4:56:19<06:02, 362.84s/it]

12250/12500 (epoch 49), train_loss = 0.17872626
model saved to small models/model.ckpt
12350/12500 (epoch 49), train_loss = 0.11343014
model saved to small models/model.ckpt
12450/12500 (epoch 49), train_loss = 0.11445989


100%|██████████| 50/50 [5:02:21<00:00, 362.83s/it]


## Test

In [93]:
#-*- coding: utf-8 -*-

from __future__ import print_function
from __future__ import unicode_literals

import tensorflow as tf

### Build Network

In [94]:
LEARNING_RATE = 0.002
DECAY_STEP = num_batches
DECAY_RATE = 0.97
DROPOUT_RATE = 0.3


net = chosungRNN(tf, 1, seq_length, vocab_size, vocab_embedded, LEARNING_RATE, num_batches, DECAY_RATE, DROPOUT_RATE, False)
print("Network Ready")

Network Ready


### Sampling

In [105]:
PRIME_TEXT = 'ㅅㅈㅎㅈ ㅁㄱ'



if net.restore():
    sampled_text = net.sample(PRIME_TEXT)
    
    sampled_text = re.sub('\<eos\>', '', sampled_text)
    print("")
    print("-- RESULT --")
    print("%s" % PRIME_TEXT)
    print(sampled_text)

Restore network from  small models/model.ckpt-12400
INFO:tensorflow:Restoring parameters from small models/model.ckpt-12400

-- RESULT --
ㅅㅈㅎㅈ ㅁㄱ
사주하잔 마고
